# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Part 1 File (CSV)
Part_1_file = pd.read_csv("Weather_data.csv")
Part_1_file.head()

,Unnamed: 0,city,latitude,Longitude,Temperature Max,Humidity,Cloudiness,Wind Speed,Country
0,0,cervo,43.6702,-7.4101,8.33,100,98,0.45,ES
1,1,mataura,-46.1927,168.8643,8.33,71,98,7.60,NZ
2,2,luderitz,-26.6481,15.1594,16.50,88,96,1.53,NaN
3,3,ushuaia,-54.8000,-68.3000,8.00,76,75,5.66,AR
4,4,castro,-24.7911,-50.0119,13.19,96,80,1.13,BR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#Store latitude and longitude into locations
locations = Part_1_file[["latitude", "Longitude"]]
humidity = Part_1_file["Humidity"].astype(float)

In [7]:
#Plot Heatmap
fig=gmaps.figure()

#create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

#add layer
fig.add_layer(heat_layer)

#Display

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#get indices for temp over 80F or 27C
index_temp = Part_1_file.index[Part_1_file['Temperature Max']>=27]
str(index_temp)


"Int64Index([  7,  14,  18,  19,  23,  27,  33,  43,  46,  51,  56,  60,  80,\n             93, 103, 104, 115, 116, 120, 121, 123, 128, 136, 144, 149, 159,\n            162, 170, 184, 186, 201, 216, 219, 221, 227, 228, 233, 237, 238,\n            239, 254, 257, 261, 264, 278, 293, 306, 308, 313, 325, 326, 331,\n            337, 351, 354, 361, 362, 376, 388, 389, 395, 399, 400, 406, 416,\n            420, 438, 445, 449, 454, 456, 468, 472, 473, 474, 517, 518, 542,\n            544, 545, 549, 550, 558, 560, 565, 566, 568, 570],\n           dtype='int64')"

In [25]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
Perfec_temp = Part_1_file.drop(index=index_temp, inplace=False)
Perfec_temp.head()

,Unnamed: 0,city,latitude,Longitude,Temperature Max,Humidity,Cloudiness,Wind Speed,Country
0,0,cervo,43.6702,-7.4101,8.33,100,98,0.45,ES
1,1,mataura,-46.1927,168.8643,8.33,71,98,7.60,NZ
2,2,luderitz,-26.6481,15.1594,16.50,88,96,1.53,NaN
3,3,ushuaia,-54.8000,-68.3000,8.00,76,75,5.66,AR
4,4,castro,-24.7911,-50.0119,13.19,96,80,1.13,BR


In [28]:
#Get indices for temp under 70F or 21C
index_temp2 = Perfec_temp.index[Perfec_temp['Temperature Max']<21]
str(index_temp2)

"Int64Index([  0,   1,   2,   3,   4,   5,   6,   9,  12,  13,\n            ...\n            557, 559, 561, 562, 563, 564, 567, 569, 571, 572],\n           dtype='int64', length=359)"

In [29]:
Perfec_temp_update = Perfec_temp.drop(index=index_temp2, inplace=False)
Perfec_temp_update.head()

,Unnamed: 0,city,latitude,Longitude,Temperature Max,Humidity,Cloudiness,Wind Speed,Country
8,8,saint-philippe,-21.3585,55.7679,26.0,69,0,8.23,RE
10,10,mahebourg,-20.4081,57.7000,25.0,88,75,6.69,MU
11,11,bambous virieux,-20.3428,57.7575,25.0,88,75,6.69,MU
21,21,cairns,-16.9167,145.7667,24.0,94,90,6.17,AU
22,22,kapaa,22.0752,-159.3190,24.0,73,40,6.17,US


In [32]:
#Get Indices for windspeed greater than 10mph
index_ws = Perfec_temp_update.index[Perfec_temp_update['Wind Speed']>10]
str(index_ws)

"Int64Index([198], dtype='int64')"

In [33]:
Perfec_ws = Perfec_temp_update.drop(index=index_ws, inplace=False)
Perfec_ws.head()

,Unnamed: 0,city,latitude,Longitude,Temperature Max,Humidity,Cloudiness,Wind Speed,Country
8,8,saint-philippe,-21.3585,55.7679,26.0,69,0,8.23,RE
10,10,mahebourg,-20.4081,57.7000,25.0,88,75,6.69,MU
11,11,bambous virieux,-20.3428,57.7575,25.0,88,75,6.69,MU
21,21,cairns,-16.9167,145.7667,24.0,94,90,6.17,AU
22,22,kapaa,22.0752,-159.3190,24.0,73,40,6.17,US


In [37]:
#Get indices for non zero cloudiness
index_cloud = Perfec_ws.index[Perfec_ws['Cloudiness'] != 0]
str(index_cloud)

"Int64Index([ 10,  11,  21,  22,  24,  25,  29,  47,  55,  57,\n            ...\n            520, 527, 528, 530, 533, 535, 537, 541, 548, 555],\n           dtype='int64', length=105)"

In [38]:
#final DF
Final_Weather = Perfec_ws.drop(index=index_cloud, inplace=False)
Final_Weather.head()

,Unnamed: 0,city,latitude,Longitude,Temperature Max,Humidity,Cloudiness,Wind Speed,Country
8,8,saint-philippe,-21.3585,55.7679,26.00,69,0,8.23,RE
52,52,andros town,24.7000,-77.7667,24.00,88,0,1.03,BS
199,199,beloha,-25.1667,45.0500,21.48,74,0,2.51,MG
206,206,saint-pierre,-21.3393,55.4781,26.00,69,0,8.23,RE
207,207,broken hill,-31.9500,141.4333,21.00,37,0,5.14,AU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = []
Final_Weather["Hotel Name"]=hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
